In this notebook, I try using the multiple time step objective function. This is given by

$$ \sum_{j=1}^{m} | x^j - \tilde{x}^j|^2,$$
where the approximate time series is defined recursively by $\tilde{x}^0 = x^0$, $\tilde{x}^j=f(\tilde{x}^{j-1}) + (g^i + g^{i+1})\frac{h}{2}$ for $j>1$. Here, $f$ will be approximated using a neural network.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import torch
from sklearn.externals import joblib


from lib.evaluation.single_column import runsteps, forced_step, step

Let's load the data.

In [ ]:
ms_obj_stepper = torch.load("../data/ml/ngaqua/model.2.torch")

data = joblib.load("../data/ml/ngaqua/time_series_data.pkl")

X = data['X']
G = data['G']
scale = data['scales']
w = data['w']

# # we need to grap the pressure field from a different path
p = xr.open_dataset("../data/raw/ngaqua/stat.nc").p.values
# t = dt * np.arange(X.shape[0])

and compute the mean.

Let's make some interpolators for the forcing as well as the observed time series

# Multiple time step objective function

In [ ]:
def plot_soln(x, y):
    
    fig, axs = plt.subplots(2,2, figsize=(8,5), sharey=True)
    qt_levs = np.arange(11)*2.5


    t_levs = np.arange(12)*25 + 275
    t_im = axs[0,0].contourf(x[:,:34].T, levels=t_levs)
    axs[0,1].contourf(y[:,:34].T, levels=t_levs)
    q_im = axs[1,0].contourf(x[:,34:].T, levels=qt_levs, extend='both')
    axs[1,1].contourf(y[:,34:].T, levels=qt_levs, extend='both')

    plt.colorbar(t_im, ax=axs[0,:].tolist())
    plt.colorbar(q_im, ax=axs[1,:].tolist())
    
    axs[0,1].set_title("Prediction")
    axs[0,0].set_title("Truth")
    
    axs[0,0].set_ylabel('sl')
    axs[1,0].set_ylabel('qt')


We train the neural network outside of this notebook because it takes a long time to run. We use a 10 time step long window to compute the loss (i.e. $m=10$).

In [ ]:
# grab a specific location
loc = [slice(None), 8, 0, slice(None)]
x = X[loc]
g = G[loc]

In [ ]:
y = runsteps(step(ms_obj_stepper), x[0], x.shape[0])
plot_soln(x, y)

As we can see the unforced model appears to approach a reasonable steady state value.

Forced stepper

In [ ]:
y = runsteps(forced_step(ms_obj_stepper, g), x[0], x.shape[0])
plot_soln(x, y)